In [1]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm
from hmmlearn.hmm import GaussianHMM
from hmmlearn.hmm import GMMHMM
from sklearn.model_selection import train_test_split

In [2]:
def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1))
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix

In [3]:
def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
    return mfcc

In [4]:
def clustering(X, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0, verbose=0)
    kmeans.fit(X)
    print("centers", kmeans.cluster_centers_.shape)
    return kmeans

In [15]:
import random

class_names = ['tôi', 'nhà', 'học', 'nhân viên', 'hà nội']
dataset = {}
dataset_train = {}
dataset_test = {}

for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = get_class_data(os.path.join("data", cname))
#     uncomment to shuffle dataset
    random.shuffle(dataset[cname])
    train_size = int(0.8*len(dataset[cname]))
    dataset_train[cname] = dataset[cname][:train_size]
    dataset_test[cname] = dataset[cname][train_size:]

# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
# print("vectors", all_vectors.shape)
# Run K-Means algorithm to get clusters
# Comment KMEANS for GMMHMM
# kmeans = clustering(all_vectors)
# print("centers", kmeans.cluster_centers_.shape)

Load tôi dataset
Load nhà dataset
Load học dataset
Load nhân viên dataset
Load hà nội dataset


In [16]:
models = {}
for cname in class_names:
    class_vectors = dataset[cname]
#     use Multinominal HMM
#     dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])
#     hmm = hmmlearn.hmm.MultinomialHMM(
#         n_components=20, random_state=0, n_iter=1000, verbose=True
#     )


    hmm = GMMHMM(
    n_components=5, n_mix = 1, random_state=42, n_iter=1000, verbose=True,
        params='mctw',
        init_params='mc',
#         startprob_prior = np.array([1.0,0.0,0.0,0.0,0.0]),
#         transmat_prior = np.array([
#             [0.7,0.3,0.0,0.0,0.0],
#             [0.0,0.7,0.3,0.0,0.0],
#             [0.0,0.0,0.7,0.3,0.0],
#             [0.0,0.0,0.0,0.7,0.3],
#             [0.0,0.0,0.0,0.0,1.0],
#         ])
    )
    hmm.startprob_ = np.array([1.0,0.0,0.0,0.0,0.0])
#     hmm.transmat_ = np.array([
#         [0.7,0.3,0.0,0.0,0.0],
#         [0.0,0.7,0.3,0.0,0.0],
#         [0.0,0.0,0.7,0.3,0.0],
#         [0.0,0.0,0.0,0.7,0.3],
#         [0.0,0.0,0.0,0.0,1.0],
#     ])

#     uncomment below line to train with full dataset
#     X = np.concatenate(dataset[cname])
#     lengths = list([len(x) for x in dataset[cname]])
#     hmm.fit(X, lengths=lengths)

    X = np.concatenate(dataset_train[cname])
    lengths = list([len(x) for x in dataset_train[cname]])
#     FOR GMMHMM: NO NEED lengths parameter
    hmm.fit(X)
    models[cname] = hmm
print("Training done")

print("Testing")

for true_cname in class_names:
    true_predict = 0
#     for O in dataset[true_cname]:
    for O in dataset_test[true_cname]:
        score = {cname : model.score(O, [len(O)]) for cname, model in models.items()}
        predict = max(score, key=score.get)
        if predict == true_cname:
            true_predict += 1
#         print(true_cname, score, predict)
    print(true_cname)
#     change dataset_test to dataset to test in full dataset
    print(f'TRUE PREDICT: {true_predict}/{len(dataset_test[true_cname])}')
    print('ACCURACY:', true_predict/len(dataset_test[true_cname]))

         1     -213020.8705             +nan
         2     -208292.9744       +4727.8962
         3     -205931.7864       +2361.1880
         4     -205176.0909        +755.6955
         5     -204858.2171        +317.8739
         6     -204664.7035        +193.5136
         7     -204455.0198        +209.6836
         8     -204277.2947        +177.7252
         9     -204155.9364        +121.3583
        10     -204059.9700         +95.9664
        11     -203942.8557        +117.1144
        12     -203766.8708        +175.9849
        13     -203618.4955        +148.3753
        14     -203545.5698         +72.9257
        15     -203471.9107         +73.6591
        16     -203417.9191         +53.9915
        17     -203385.8781         +32.0411
        18     -203355.2250         +30.6530
        19     -203334.9034         +20.3217
        20     -203319.1147         +15.7886
        21     -203297.8306         +21.2841
        22     -203254.6330         +43.1976
        23

Training done
Testing
tôi
TRUE PREDICT: 18/20
ACCURACY: 0.9
nhà
TRUE PREDICT: 14/15
ACCURACY: 0.9333333333333333
học
TRUE PREDICT: 10/11
ACCURACY: 0.9090909090909091
nhân viên
TRUE PREDICT: 21/21
ACCURACY: 1.0
hà nội
TRUE PREDICT: 10/11
ACCURACY: 0.9090909090909091


In [13]:
len(dataset_test['hà nội'])

11

In [14]:
len(dataset_train['hà nội'])

44

In [17]:
models

{'tôi': GMMHMM(algorithm='viterbi', covariance_type='diag',
        covars_prior=array([[[-1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
          -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
          -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
          -1.5, -1.5, -1.5, -1.5, -1.5, -1.5]],
 
        [[-1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
          -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5...
        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.]]]),
        means_weight=array([[0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
        min_covar=0.001, n_components=5, n_iter=1000, n_mix=1, params='mctw',
        random_state=42, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
        verbose=True,
        weights_prior=array([[1.],
        [1.],
        [1.],
        [1.],
 

In [18]:
import pickle 
with open("models.pkl", "wb") as file:
    pickle.dump(models, file)